In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import datetime
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
#tf.truncated_normal(shape, mean, stddev) :shape表示生成张量(kernel)的维度，mean是均值，stddev是标准差
#shape [height,width,previous_channel,forward_channel]
C1_filter = tf.Variable(tf.truncated_normal([5,5,1,6], stddev = 0.1 ) )
C1_biases = tf.Variable(tf.zeros([6]))
C3_filter = tf.Variable(tf.truncated_normal([5,5,6,16], stddev = 0.1 ))
C3_biases = tf.Variable(tf.constant(1.0, shape=[16]))
C5_filter = tf.Variable(tf.truncated_normal([5,5,16,120], stddev = 0.1))
C5_biases = tf.Variable(tf.constant(1.0, shape=[120]))

OUTPUT_weights = tf.Variable(tf.truncated_normal([84,10], stddev = 0.1))
OUTPUT_biases = tf.Variable(tf.constant(1.0, shape=[10]))

#使用了 None表示，shape不確定维度時，以任意的維度做輸入。name為Graph抓的參數名稱
xs = tf.placeholder(tf.float32, [None,784], name = "input_image")
#[batch_size,Height,Width,灰階為1]
xs2 = tf.reshape(xs,[-1,28,28,1])
xs3 = tf.image.per_image_standardization(xs2)

#[filter_height, filter_width, in_channels, out_channels]
C1 = tf.nn.conv2d(xs3, C1_filter, [1,1,1,1], padding = "SAME" )  ##C1: 6@28X28 
C1 = tf.nn.relu(C1 + C1_biases)
print("C1:     ", C1.get_shape().as_list())

#用2X2的格子，長跟高都間隔2去跳
S2 = tf.nn.max_pool(C1, [1,2,2,1], [1,2,2,1], padding = "VALID")  ##S2: 6@14X14
print("S2:     ", S2.get_shape().as_list())

C3 = tf.nn.conv2d(S2, C3_filter, [1,1,1,1], padding = "VALID")    ##C3: 16@10X10
C3 = tf.nn.relu(C3 + C3_biases)
print("C3:     ", C3.get_shape().as_list())

S4 = tf.nn.max_pool(C3, [1,2,2,1], [1,2,2,1], padding = "VALID")   ##S4: 16@5X5
print("S4:     ", S4.get_shape().as_list())

C5 = tf.nn.conv2d(S4, C5_filter, [1,1,1,1], padding = "VALID")     ##C5: 120@1X1
C5 = tf.nn.relu(C5 + C5_biases)
print("C5:     ", C5.get_shape().as_list())

shape = C5.get_shape().as_list()
#判斷C5的shape多少，做後面的reshape
#[batch_size,height,width,channel]
s1 = shape[1] * shape[2] * shape[3]
reshape = tf.reshape(C5,[-1,s1])
F6_weights = tf.Variable(tf.truncated_normal([s1,84], stddev = 0.1))
F6_biases = tf.Variable(tf.constant(1.0, shape = [84]))
print("reshape:", reshape.get_shape().as_list())

F6 = tf.nn.relu(tf.matmul(reshape, F6_weights) + F6_biases)
print("F6:     ", F6.get_shape().as_list())

y = tf.matmul(F6, OUTPUT_weights) + OUTPUT_biases
print("OUTPUT: ", y.get_shape().as_list(), "\n\n")

#保存y，以便在預測的時候使用
tf.add_to_collection('pred_network', y)

# Define loss and optimizer

ys = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=ys))
train_step = tf.train.GradientDescentOptimizer(0.06).minimize(cross_entropy)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
train_accuracy = []
test_accuracy = []
start = datetime.datetime.now()
#建立saver
saver = tf.train.Saver()
for i in range(20000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
    if i % 2000 == 0:
        
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(ys, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        acc_val = sess.run(accuracy, feed_dict={xs: mnist.train.images, ys: mnist.train.labels})
        train_accuracy.append(acc_val)
        
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(ys, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        acc_val2 = sess.run(accuracy, feed_dict={xs: mnist.test.images, ys: mnist.test.labels})
        test_accuracy.append(acc_val2)
        print("Train Accuracy=%f, Test Accuracy=%f" % (acc_val,acc_val2))
end = datetime.datetime.now()
print("總訓練時間: %d s" % (end-start).seconds)
plt.axes([0,0,1,1])
plt.plot(train_accuracy, "-r" ,  label = "train_accuracy")
plt.plot(test_accuracy, "-k" ,  label = "test_accuracy")
plt.legend()

In [ ]:
#儲存model
saver.save(sess,"./model/model.ckpt")

"""
model.ckpt.meta文件保存了TensorFlow計算圖的架構，也就是整個類神經網路的架構
model.ckpt文件保存了TensorFlow程序中每一個變數的取值
checkpoint文件保存了一個目錄下所有的model文件列表
"""

sess.close()
exit()